In [1]:
import numpy as np
import pandas as pd
import os
import shutil

# Test EM with sparsity

In this part we generate a fake classifier that gets oracle results but outputs a 1000 sparsity signal. We will use it with the algorithm to check that our code works correctly.

In [2]:
dataset_path="/home/jkipen/raid_storage/ProtInfGPU/data/20642_Prot/binary/";
classifier_name="SparsityTest";

In [3]:
#Some functions to simplify 
def create_folder(folder_path):
    if not os.path.exists(folder_path):  
        os.makedirs(folder_path)
def create_classifier_folders(ds_path,classifier_name):
    classifier_path=ds_path+"/"+classifier_name;
    create_folder(classifier_path);
    create_folder(classifier_path+"/CrossVal"); #Create subfolders
    create_folder(classifier_path+"/Common");
def copy_all_files(src_folder, dst_folder):
    """Copies all files from src_folder to dst_folder, creating dst_folder if it doesn't exist."""
    #os.makedirs(dst_folder, exist_ok=True)  # Ensure the destination folder exists
    for file_name in os.listdir(src_folder):
        src_file = os.path.join(src_folder, file_name)
        dst_file = os.path.join(dst_folder, file_name)
        if os.path.isfile(src_file):  # Ensure it's a file before copying
            shutil.copy2(src_file, dst_file)
def copy_oracle_info_to_classifier(ds_path,classifier_name):
    #copy_all_files(os.path.join(ds_path, "Oracle", "Common"),
    #               os.path.join(ds_path, classifier_name, "Common"))
    copy_all_files(os.path.join(ds_path, "Oracle", "CrossVal"),
                   os.path.join(ds_path, classifier_name, "Crossval"));
    

In [13]:
create_classifier_folders(dataset_path,classifier_name);
copy_oracle_info_to_classifier(dataset_path,classifier_name);

In [6]:
trueIds=np.fromfile(dataset_path+"Common/trueIds.bin",dtype=np.uint32)
trueIds

array([     0,      0,      0, ..., 152290, 152290, 152290], dtype=uint32)

In [7]:
nSparsity=10;
TopNScoresAux=np.zeros((len(trueIds),nSparsity),dtype=np.float32);
TopNScoresIdAux=np.zeros((len(trueIds),nSparsity),dtype=np.uint32);

In [8]:
TopNScoresIdAux[:,0]=trueIds; #These top scores dont have other idxs for sparsity values and are not ordered
TopNScoresAux[:,0]=1;
auxMat=np.tile(np.arange(nSparsity), (len(trueIds), 1));
TopNScoresIdAux= (trueIds.reshape(-1, 1)+auxMat)%(np.max(trueIds)+1) # here we create
sort_indices = np.argsort(TopNScoresIdAux, axis=1)
rows = np.arange(TopNScoresIdAux.shape[0])[:, None]  # Row indices for broadcasting
TopNScoresAux[:] = TopNScoresAux[rows, sort_indices]
TopNScoresIdAux[:] = TopNScoresIdAux[rows, sort_indices]

In [13]:
TopNScoresAuxF=TopNScoresAux.flatten();
TopNScoresIdAuxF=TopNScoresIdAux.flatten();

In [24]:
TopNScoresAuxF.astype(np.float32).tofile(dataset_path+classifier_name+"/Common/TopNScores.bin")
TopNScoresIdAuxF.astype(np.uint32).tofile(dataset_path+classifier_name+"/Common/TopNScoresId.bin")

In [49]:
nSpar=np.array([10],dtype=np.uint32 )
nSpar.tofile(dataset_path+classifier_name+"/Common/nSparsity.bin")

In [28]:
TopNScoresIdAuxF[-20:]
#TopNScoresIdAux[152289000,:]

array([     0,      1,      2,      3,      4,      5,      6,      7,
            8, 152290,      0,      1,      2,      3,      4,      5,
            6,      7,      8, 152290])

In [26]:
aux=np.fromfile(dataset_path+classifier_name+"/Common/TopNScoresId.bin",dtype=np.uint32)

In [29]:
aux[-20:]

array([     0,      1,      2,      3,      4,      5,      6,      7,
            8, 152290,      0,      1,      2,      3,      4,      5,
            6,      7,      8, 152290], dtype=uint32)

In [9]:
TopNScoresIdAux

array([[     0,      1,      2, ...,      7,      8,      9],
       [     0,      1,      2, ...,      7,      8,      9],
       [     0,      1,      2, ...,      7,      8,      9],
       ...,
       [     0,      1,      2, ...,      7,      8, 152290],
       [     0,      1,      2, ...,      7,      8, 152290],
       [     0,      1,      2, ...,      7,      8, 152290]])

# Generating Whatprot format datasets

In [2]:
whatprot_path= "/../ext/whatprot/cc_code/bin/release/whatprot"
common_path_datasets= "/raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/"
dye_seqs_path=common_path_datasets + "DyeSeqs.tsv"
params_path = common_path_datasets + "seqparams_atto647n_x3.json"
n_samples_per_dataset=10000000; #5 Gb per dataset

In [19]:
n_datasets=10;
for i in range(n_datasets):
    rad_path = common_path_datasets+ "MultiDatasets/radiometries_"+str(i)+".tsv";
    true_ids_path = common_path_datasets+ "MultiDatasets/true-ids_"+str(i)+".tsv";
    command = "." +whatprot_path + " simulate rad -t 10 -g "+str(n_samples_per_dataset) +" -P "+ params_path+" -S "+dye_seqs_path+" -R "+rad_path+" -Y "+true_ids_path;
    print(command)
    os.system(command)

./../ext/whatprot/cc_code/bin/release/whatprot simulate rad -t 10 -g 10000000 -P /raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/seqparams_atto647n_x3.json -S /raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/DyeSeqs.tsv -R /raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/MultiDatasets/radiometries_0.tsv -Y /raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/MultiDatasets/true-ids_0.tsv
Finished basic setup (9.97558e-05 seconds).
Read 152291 dye seqs (0.111768 seconds).
Finished generating 9497843 radiometries (66.2802seconds).
Finished saving results (330.784 seconds).
Total run time: 397.176 seconds.
./../ext/whatprot/cc_code/bin/release/whatprot simulate rad -t 10 -g 10000000 -P /raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/seqparams_atto647n_x3.json -S /raid/jkipen/ProtInfGPU/data/WhatprotGen/WholeProteomeTests/ProbeamPaperParams/DyeS

In [20]:
all_true_ids=[]
for i in range(10):
    df = pd.read_csv(common_path_datasets+ "MultiDatasets/true-ids_"+str(i)+".tsv", sep="\t");
    all_true_ids.append(df.to_numpy().flatten());

In [27]:
#all_true_ids=np.concatenate(all_true_ids)
u,counts=np.unique(all_true_ids,return_counts=True)
print(np.max(counts))
print(np.min(counts))
print(len(u))

889
390
152291


This means that with these datasets we can create an extra dataset with samples up to 390 of each dye seq!. Now lets get those datasets together and pick 100 of them!


In [14]:
dfs = [] #Vibe coding
for i in range(10):
    dfr = pd.read_csv(common_path_datasets+ "MultiDatasets/radiometries_"+str(i)+".tsv", sep="\t", header=None, skiprows=3); #Skipping metadata rows
    dft = pd.read_csv(common_path_datasets+ "MultiDatasets/true-ids_"+str(i)+".tsv", sep="\t", header=None, skiprows=1); #Skipping metadata rows
    dfr['true_ids'] = dft[0]
    dfs.append(dfr)
    
big_df = pd.concat(dfs, axis=0, ignore_index=True)
result_df = big_df.groupby('true_ids', group_keys=False).apply(lambda group: group.head(100))

In [15]:
result_df.to_csv(common_path_datasets+"aux.csv")

KeyboardInterrupt: 

In [18]:
# Parameters for the header of the output file
#Vibe oding
radiometries_df = result_df.drop(columns=['true_ids'])
true_ids_series = result_df['true_ids']

# Get the number of reads (rows) from the filtered data
num_reads = len(result_df)

# Save the final radiometries file with custom header
num_timesteps = 10   # Given constant: number of timesteps
num_channels = 3     # Given constant: number of channels



# Save the final true IDs file with custom header (only number of reads)
true_ids_output_file = common_path_datasets+"true-ids.tsv"
with open(true_ids_output_file, "w") as f:
    f.write(f"{num_reads}\n")
    true_ids_series.to_csv(f, sep="\t", index=False, header=False)


In [19]:
radiometries_output_file = common_path_datasets+"radiometries.tsv"
with open(radiometries_output_file, "w") as f:
    f.write(f"{num_timesteps}\n")
    f.write(f"{num_channels}\n")
    f.write(f"{num_reads}\n")
    radiometries_df.to_csv(f, sep="\t", index=False, header=False)